In [72]:
import numpy as np
import pandas as pd

# Loading flights data per month from csv into dataframe
# Data from https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236
# Data provided in per month blocks, selected month blocks were filtered by 
# 'Washington' i.e. all flights that arrived or departed from airports that
# were in the database that were in Washington. Nov. 2018 is the latest flight
# data released.
dec_2017_df = pd.read_csv('data-raw/dec_2017_T_ONTIME_REPORTING.csv', index_col=False)
jan_2018_df = pd.read_csv('data-raw/jan_2018_T_ONTIME_REPORTING.csv', index_col=False)
feb_2018_df = pd.read_csv('data-raw/feb_2018_T_ONTIME_REPORTING.csv', index_col=False)
mar_2018_df = pd.read_csv('data-raw/mar_2018_T_ONTIME_REPORTING.csv', index_col=False)
apr_2018_df = pd.read_csv('data-raw/apr_2018_T_ONTIME_REPORTING.csv', index_col=False)
may_2018_df = pd.read_csv('data-raw/may_2018_T_ONTIME_REPORTING.csv', index_col=False)
jun_2018_df = pd.read_csv('data-raw/jun_2018_T_ONTIME_REPORTING.csv', index_col=False)
jul_2018_df = pd.read_csv('data-raw/jul_2018_T_ONTIME_REPORTING.csv', index_col=False)
aug_2018_df = pd.read_csv('data-raw/aug_2018_T_ONTIME_REPORTING.csv', index_col=False)
sep_2018_df = pd.read_csv('data-raw/sep_2018_T_ONTIME_REPORTING.csv', index_col=False)
oct_2018_df = pd.read_csv('data-raw/oct_2018_T_ONTIME_REPORTING.csv', index_col=False)
nov_2018_df = pd.read_csv('data-raw/nov_2018_T_ONTIME_REPORTING.csv', index_col=False)

In [73]:
# Combining separate month dataframes into one year long data frame
flights = pd.concat([dec_2017_df, jan_2018_df, feb_2018_df, mar_2018_df, 
                     apr_2018_df, may_2018_df, jun_2018_df, jul_2018_df,
                     aug_2018_df, sep_2018_df, oct_2018_df, nov_2018_df])

In [74]:
# Load in weather data from https://www.ncdc.noaa.gov/cdo-web/datatools/lcd
weather_2018 = pd.read_csv('data-raw/weather_2018.csv', index_col=False) # data for all of 2018
weather_dec_2017 = pd.read_csv('data-raw/weather_dec_2017.csv', index_col=False) # data for December 2017

# Combine into one dataframe for weather for Dec 2017 - November 2018
weather = pd.concat([weather_dec_2017, weather_2018])

In [24]:
# Check column data types
print(flights.dtypes)

# I notice a column that wasn't described as in the dataset on the BTS website so I drop it
flights = flights.drop('Unnamed: 31', axis = 1)

# Double check it's dropped
print(flights.columns)

YEAR                       int64
QUARTER                    int64
MONTH                      int64
DAY_OF_MONTH               int64
DAY_OF_WEEK                int64
FL_DATE                   object
OP_UNIQUE_CARRIER         object
ORIGIN_AIRPORT_ID          int64
ORIGIN_AIRPORT_SEQ_ID      int64
ORIGIN_CITY_MARKET_ID      int64
ORIGIN                    object
ORIGIN_CITY_NAME          object
DEST_AIRPORT_ID            int64
DEST_AIRPORT_SEQ_ID        int64
DEST_CITY_MARKET_ID        int64
DEST                      object
DEST_CITY_NAME            object
CRS_DEP_TIME               int64
DEP_TIME                 float64
DEP_DELAY                float64
DEP_DELAY_NEW            float64
DEP_DEL15                float64
CANCELLED                float64
DIVERTED                 float64
AIR_TIME                 float64
DISTANCE                 float64
CARRIER_DELAY            float64
WEATHER_DELAY            float64
NAS_DELAY                float64
SECURITY_DELAY           float64
LATE_AIRCR

In [71]:
# Find flights only leaving SeaTac
flights = flights[flights.ORIGIN == 'SEA']

In [137]:
# Create new series that represents the hour a flight happened in
dep_times = flights.CRS_DEP_TIME.apply(str)
hours = []
for time in dep_times:
    if len(time) == 1 or len(time) == 2:
        hours.append('00')
    elif len(time) == 3:
        hours.append('0' + time[0])
    else:
        hours.append(time[:2])

# Create new series for day of month in dd format
day_of_month = flights.DAY_OF_MONTH.apply(str)
days = []
for day in day_of_month:
    if len(day) == 1:
        days.append('0' + day)
    else:
        days.append(day)

# Create new series for month in mm format
month = flights.MONTH.apply(str)
months = []
for m in month:
    if len(m) == 1:
        months.append('0' + m)
    else:
        months.append(m)

# Create new variable in format yyyymmddhh to allow join with weather data
flights['join_time'] = flights.YEAR.apply(str) + months + days + hours

# Change 'join_time' type to int64
flights.join_time.astype('int64')

In [187]:
# Get the join_time value for each hour
join_time = list(flights.groupby('join_time').groups.keys())

# Compute the average flight delay for each join_time hour
avg_delay = flights.groupby('join_time')['DEP_DELAY'].aggregate('mean')

# Create a dataframe with the join_time and avg_delay
flight_delays = pd.DataFrame({'join_time':join_time,
                              'avg_delay':list(avg_delay)})

In [191]:
# Save to .csv to use to combine with weather data
flight_delays.to_csv('data/hourly_avg_flight_delays.csv', index=False)